医療データ解析(１)

# 概要
* 医療データはcsvもしくはRDBのフォーマットで提供されることが多い
* データ処理を効率的に行うため、それらのフォーマットのデータを処理する方法に触れる
  * Pandas (pythonのライブラリ) の初歩的な操作に触れる
  * Pandasの操作を踏まえて、リレーショナルデータベース (RDB) の初歩的な操作に触れる

## データ処理知識の重要性
データ解析を行う際に利用する大規模なデータ（例えば病院のカルテデータ、レセプトデータなど）は、大抵業務用データウェアハウス (DWH) から抽出し提供されています。そのデータベースの多くはリレーショナルデータベース (RDB) です。

そのため、提供されるデータはcsvなど一般的なフォーマットの場合もあるが、データベースの出力ファイルを渡される場合が多い。その場合は、データにアクセスするためにはRDBへリストアしSQL言語などを用いて問い合わせる必要があります。

実際の業務から得られたデータは、そのまま解析や機械学習に使えるほど綺麗には整えられてはいません。

データ分析において、データサイエンティストが最も時間を費やし、かつ最も楽しめない業務としてデータの整理やノイズや誤字などを除去し綺麗にするデータクリーニング（クレンジング）作業が挙げられています（2017 Data Scientist Report, CrowdFlower）。

このクリーニング作業は、どのようなデータが誤っているのかを見分けることができなければならないため、作業者がその分野の知識を有していなければならない。

医療データは要配慮個人情報であり、多くの場合保管やアクセスの条件が厳しいため、外部の力を借りるハードルがやや高く、そのため研究者のチーム内でデータ処理の多くの部分を賄う必要があります。

提供されたデータを処理するとき、巨大なデータの扱い方を知っているか知らないかでは処理時間に大きな差が出てきます。

発展著しい医療情報分野において、研究の生産性を高めるためにもデータ処理の知識が重要となります。


## Pandas (Python)
Pandasはpythonの外部ライブラリでも著名なものの一つであり、大規模なデータセットを処理するための便利な機能を提供しています。

listやdictなどの組み込み型を駆使して行うデータ操作よりも、比較的人間の直感に近い操作を提供しています。

使用者が多くWeb上にドキュメントやヘルプも豊富なので、やりたいことを実現するコードが既に誰かが公開していたり、エラーに直面してもエラーコードを検索すれば解決法やヒントが見つかりやすいので、何はともあれpythonでデータ処理を行うのであればpandasを用いることになると思います。

## リレーショナルデータベース (RDB)
データベースとは、複数による共同利用を意図して、データを組織的に、効率的に、永続的に蓄積し整理がなされたデータ群のことであり、これを管理することを目的としたシステムがデータベースシステムです。

複数人からのアクセスを意図しているため、データベースシステムは、データの持ち方やデータの操作の手続きというものがあらかじめ規定されています（データモデル）。使用者とデータベースシステムの間に立ち、使用者の指示に対してその規定に従うような具体的な操作を行うことによって、両者の仲立ちを行うのがデータベースマネジメントシステム（DBMS）です。

現在世の中によく普及しているのが、データモデルとしてリレーション（関係）を採用し（しばしば例えに表が持ち出されます）、データベースをリレーションの集合とみなすリレーショナルデータベース (RDB, もしくは関係データベース）で、その管理システムがリレーショナルデータベースマネジメントシステム (RDBMS)です。RDBの設計にはリレーションの正規化などの概念があります（この演習では触れませんが、データベース設計などに興味がある方は、「データベースシステム(北川博之)」、「リレーショナルデータベース入門 (増永良文)」などを手に取ってみてください）。

多くの業務システムはRDBを採用しています。そのため、自治体や医療機関から提供される医療データもまたそれに準じた形式であることが多いです。

この時間は、SQLiteというRDBMSを用いてRDBの操作方法についても触れます。

# 基本操作とその比較

以下のオープンデータを用いて操作の基本を学びます。
ブラウザからダウンロードするか、colabのセルに以下のコマンドを打ち込んでデータをダウンロードしてください。回線が細いのか数十秒かかります。

https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
よりcsvを取得してみましょう。

https://opendata.ecdc.europa.eu/covid19/casedistribution/csv

In [ ]:
!wget -c https://opendata.ecdc.europa.eu/covid19/casedistribution/csv

ファイルの内容を確認してみましょう。

In [ ]:
%%bash
# ファイル名がcsvのみで誤解を招きやすので名前変更
mv csv eu.csv
# ファイルの内容確認
head -n 10 eu.csv
# ファイルの行数確認
wc -l eu.csv

## Pythonの基本機能の場合

このcsvファイルを操作する方法について見ていきます。

まず、pythonの基本的な機能だけでcsvを操作してみましょう。

In [ ]:
# csvの読み込み
import csv

with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  # ヘッダ行
  header = next(csv_reader)
  display(header)

csvファイルは先頭に各列の名称を記録するヘッダ行が存在する場合があります。これは1行とは限らず、複数行コメント行が続くこともあるため、確認する必要があります。

今回はヘッダ行1行のみであることがわかったため、読み込んだ直後に１行をヘッダとして読み取ります。
その後、残りの行をforループで処理することによって各行への処理を行います（下の例は1ループ目でbreakすることで中断しています）。


In [ ]:
# 各行を処理する場合
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)
  
  for row in csv.DictReader(f, header):
    # 以下、行毎の処理を記述
    print('----Row----')
    print(row) # rowに各行の内容が辞書型で入っている
    print('----')
    print(row['dateRep']) # 各列の名称でアクセス
    print(row['deaths'])
    print(row['countriesAndTerritories'])
    break

Jupyter notebookではマジックコマンドとして`%time`を行頭に付けるとその行の実行時間を表示してくれます。

Google colabではセルの左側にセル全体での所要時間が表示されます。

In [ ]:
import time
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)
  %time time.sleep(2)
  for row in csv.DictReader(f, header):
    print('----Row----')
    print(row) # rowに各行の内容が辞書型で入っている
    print('----')
    print(row['dateRep']) # 各列の名称でアクセス
    print(row['deaths'])
    print(row['countriesAndTerritories'])
    break

In [ ]:
%%prun
import time
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)
  %time time.sleep(2)
  for row in csv.DictReader(f, header):
    print('----Row----')
    print(row) # rowに各行の内容が辞書型で入っている
    print('----')
    print(row['dateRep']) # 各列の名称でアクセス
    print(row['deaths'])
    print(row['countriesAndTerritories'])
    break

上の例ではcsvファイルの読み込みと、time.sleep(2)関数の時間を計測しています。後者では指示通りに2秒待機しているので2番目の計測結果のWall (clock) timeが2 sになっています。

セルの頭にマジックコマンドとして`%%time`をつけるとセルの実行時間を表示してくれます。

次の例として、csvファイルの'cases'列の要素の合計値を計算し、その実行時間を計測してみましょう。

次の例では単純に1行ずつ読み込み、cases列の値を取得して、加算するという方法をとっています。


In [ ]:
%%time
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)
  sum_cases = 0
  for row in csv.DictReader(f, header):
    sum_cases = sum_cases + row['cases']

おそらくTypeErrorが起きたと思います。これはこの読み取り方法だとcsvファイルの各要素はstr (文字列) として扱うためで、int (整数型) の計算はできないからです。

なおエラーメッセージの最下段に'Search stack overflow'というポップが出ているともいます。これをクリックすると、このエラーメッセージについてstackoverflowというコミュニティサイトに限定して検索した結果に飛ぶので、問題解決のために有用です。

ここでは解決のために、strをintとして扱うためにint(文字列)のようにします。



In [ ]:
%%time
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)
  sum_cases = 0
  for row in csv.DictReader(f, header):
    sum_cases = sum_cases + int(row['cases'])
  display(sum_cases)

この例では約6万行の処理に345 msかかったとわかります。

続いてもう一つ別の列も処理してみましょう。

In [ ]:
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)

  sum_cases = 0
  sum_deaths = 0
  for row in csv.DictReader(f, header):
    sum_cases = sum_cases + int(row['cases'])
  for row in csv.DictReader(f, header):
    sum_deaths = sum_deaths + int(row['deaths'])

  display(sum_cases)
  display(sum_deaths)
  # pythonではforは次のようにリスト内包表記でも書くことができ、こちらの方が処理速度があがることが多い

冒頭でファイルの頭数行を確認した通り確かにこの列には数値が入力されていましたが、合計値がおかしいことになっています。

これは'cases'のループでファイルを終わりまで読み込んでしまい、'deaths'のループでは何も読み込むことができなかったためです。

これを解決するためには、ファイルポインタの位置をseekメソッドによってファイルの先頭に戻す必要があります。

In [ ]:
with open('eu.csv', 'r') as f: # ここでファイルハンドルを取得しています
  csv_reader = csv.reader(f)
  header = next(csv_reader)

  sum_cases = 0
  sum_deaths = 0
  for row in csv.DictReader(f, header):
    sum_cases = sum_cases + int(row['cases'])

  f.seek(0) # ファイルポインタの位置を先頭に戻します
  header = next(csv_reader) # 先頭に戻ったのでヘッダ行を読むか、もしくは既に上で確保しているので単にnext(csv_reader)として捨てます。
  for row in csv.DictReader(f, header):
    sum_deaths = sum_deaths + int(row['deaths'])

  display(sum_cases)
  display(sum_deaths)

もしくは、単純に同じループの中で処理するという手もあります。

In [ ]:
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)

  sum_cases = 0
  sum_deaths = 0
  for row in csv.DictReader(f, header):
    sum_cases = sum_cases + int(row['cases'])
    sum_deaths = sum_deaths + int(row['deaths'])
  display(sum_cases)
  display(sum_deaths)

いずれにせよ、ファイルを上から1行ごとに処理していく、というのは手間です。
最初にcsvファイルを走査して、各行入ったリストを作成しそれを操作するというのも見通しが悪く、ファイルポインタというコンピュータシステム側に近い動作を意識しながら操作するというのも、本来やりたいこととは別の作業にリソースを取られている感じがします。

## pandasの場合

では続いてpadasによる操作を行なってみます。pandasでは、もう少し抽象的な操作が可能になります。

In [2]:
# colabではデフォルトで使用できます　自分のPCなどではpip install pandasなどとしてインストールする必要があります
# 使用頻度が高いのでpandasといちいち打つのは手間です　なのでpdで使えるようにします
import pandas as pd 
import numpy as np

In [3]:
eu = pd.read_csv('eu.csv', header=0) # ヘッダ行の有無や位置を指定することができます　しなくてもうまく対応してくれることがあります

In [ ]:
display(eu)

列名は日本語や記号も用いることができます。しかし、列名（ラベル）を指定する操作で、うまく認識されないことがあるため、できるだけ#, @, &, $, !などの記号は避け、英数字とアンダースコアを用いると良いと思います。


### データフレームの情報を表示する
pandas.DataFrameの情報を表示するためのメソッドがいくつか存在します。
データフレームの形状(行数・列数)というった一般的な情報や、要素の型をどう認識しているか、使用メモリ、各列の統計情報などを表示させることができます。

データを概観する事は、各列のカーディナリティ（列の値の種類）、欠損値の有無など分析やデータクリーニングなどを行う上でも重要な情報を与えてくれるので、データを入手したらまず行なってみましょう。

DataFrame.info()メソッドはDataFrameの形状や、各列のラベル、非欠損値のカウント情報、pandasが認識している各列のデータのタイプなどの情報を表示することができる。

これによって、例えばcsvファイルではIDとして542など文字列として格納されていたものが542.0など浮動小数点型 (float) として扱われてしまい、文字列のつもりで操作するとエラーを吐くといった状況になります。

In [ ]:
eu.info()

#### Google colab環境のスペック


In [ ]:
!df -h

In [ ]:
!cat /proc/meminfo

In [ ]:
!lscpu

ここで実際のcsvファイルの先頭をもう一度チェックしてみましょう。

In [ ]:
!head -n 3 eu.csv

In [ ]:
eu.head(10)

popData2019が浮動小数点型として扱われるべきかは迷うところです。

次にDataFrame.describe()メソッドを試してみましょう。

In [ ]:
eu.describe()

describe()メソッドは各列についての簡単な統計情報を表示してくれます。

popDataの値が大きいため指数表記になっています。pandasの表示についての設定を変更してみましょう。

推奨値などはないため、適当に状況にあわせて変更してください。

In [ ]:
pd.options.display.float_format = '{:.2f}'.format # 項目を直接指定し変更する場合
pd.set_option('display.max_columns', 50) # 正規表現で項目を検索し、マッチしたものを変更する場合
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 100)
pd.options.display.max_rows = 50

In [ ]:
eu.describe()

casesおよびdeathsのminがマイナスになっているのが気になります。時間があれば各自確認してみてください。

続いてmissingnoライブラリを用いて、NaN (非値)の分布状況を可視化してみましょう。

In [ ]:
import missingno as msno
msno.matrix(df=eu, filter='bottom', figsize=(12,9), color=(0.5,0,0))

### データ操作

ではcsvの時と同じように、各行、各列の操作をしてみます。

pandasはさまざまな方法で表の要素にアクセスできるため混乱しやすいです。基本的にはlocを知っていれば大体の事は足ります。

| | インデックスによるアクセス | ラベルによるアクセス |
|--|:--:|:--:|
| 単一要素アクセス| iat  | at |
| 複数要素アクセス| iloc | loc |

その他、eu[0:1]のような'スライス'表記によってもアクセスできます。

start:stop:stepを意味します。step=1はしばしば省略されます。

In [ ]:
#　スライス
a = [0, 1, 2, 3, 4, 5]
display(a[0:1])
display(a[1:2])
display(a[0:2])
display(a[:2])
display(a[0:6:1])
display(a[0:6:2])
display(a[:6:2])

In [ ]:
# 1つまでであればdisplayなしでその中身が表示できる
eu.iat[0, 1]

In [ ]:
eu.at[0, 'day'] # 行のラベルが数値で、結果的にインデックスと同じになっている

In [ ]:
# 1行目の指定
r0 = eu[0:1]
display(r0)

In [ ]:
# 複数行の指定
eu[0:3]

In [ ]:
# 以下の式は同じ結果を返します
display(eu.loc[[0, 1], :])
display(eu.loc[[0, 1], ])
display(eu.loc[[0, 1]])
display(eu[0:2])

In [ ]:
%%time
# 列の指定
a = eu[['cases']]
display(a)

In [ ]:
%%time
# 列の指定
eu[['cases', 'deaths']]

In [ ]:
# 列の指定(loc)
eu.loc[:, ['cases', 'deaths']]

In [ ]:
# 複数行、複数列の指定
display(eu.loc[3:5, ['cases']]) # 3:5はラベル扱い
display(eu.iloc[3:6, 4:5]) # 3:6はインデックス番号のスライス扱い
display(eu.iloc[3:6, 4:6]) # 3:6はインデックス番号のスライス扱い

In [ ]:
%%time
# 特定の条件を持つレコードの検索　pandasが読み込みの際にcasesを整数型として自動的に設定したので大小比較が可能
eu.query(' cases < 60 ')

In [ ]:
# NaNを持つ行、列の削除
# このデータに対して妥当な操作かはさておき、DataFrameにはNaNを持つ行もしくは列を簡単に削除するメソッドが存在する
eu_drop = eu.dropna(how='any', axis='rows')
# 削除後再確認してみる
msno.matrix(df=eu_drop, filter='bottom', figsize=(12,9), color=(0.5,0,0))

## SQLite (RDB) の場合

RDBを操作するためには、SQL言語を使います。SQL言語はなるべくコマンド文を英文に似せるように設計されています。Google colab.はSQLの予測補完にも対応しているので活用してください。

pythonにはsqliteライブラリが入っており、colabのjupyter notebookではsqlコマンドをセルから実行することができます (自前の環境ではipython-sqlやsqlalchemyライブラリをインストールする必要があるかもしれません)。

マジックコマンドとして%sqlを行頭につけるとsqlコマンドとして解釈されます。

%%sql だとセル全体がsqlとして解釈されます。


### CSVのインポート、テーブルの作成、レコードのインサート
操作のテストとして、先のcsvファイルをもとにSQLite3を用いてテーブル（表）を作成してみましょう。

*sqlite3のドットコマンドが利用できれば.importでcsvを簡単にインポートすることができるのですが、ここではcsvからインサートする手段を取ります。*

In [ ]:
%load_ext sql
# 接続先データベースとしてtmp.sqlite3ファイルを指定しています　存在しない場合は新規作成されます。
%sql sqlite:///tmp.sqlite3

#### テーブルの作成
まずcsvの内容を受け取るテーブルを作ります。
```
CREATE TABLE テーブル名 (列名1 型, 列名2 型, ...);  # sql文は;で文の終わりを示します。
```
のようなコマンドでテーブルを作成できます。
'テーブル名'という名前のテーブルを作り、列は'列名1', '列名2', ...、という意味です。型はその列に格納されるべき型を制限します。例えば整数型（INTEGER）が設定されている列に文字列をインサートするような操作の場合、整数型に変換を試みたり、エラーが返されたります。



格納できる型が限定されていることは様々なメリットをもたらしますが、データの整合性を保つことに有用なのもその一つです。大小比較することが明白な列などはあらかじめ数値の型を指定しておいた方が良いでしょう。

どのような型が使えるのか、違反があった場合は、変換するのか、エラーを返すのか、無視するのかなどはシステムによって異なります。SQLの規定に従っていればどのように実現するように設計するかは自由なので、基本操作にはまず違いはありませんが、SQLにも言うなれば方言があります。今回使っているSQliteのSQLと、他の有名なRDBMS、例えばPostgreSQLのSQL, MariaDBのSQL、MicrosoftのSQL serverのSQLは微妙に文法などが異なるので、違うシステムに対してはコマンドに互換性があるかを確認する必要があります。

SQLiteでは、テーブルを作るときに列に型を指定した場合、違反する型がインサートされようとした場合、まず変換を試みますが、変換できない場合はそのまま格納するという動作になります。

SQliteではTEXT型（文字列）、INTEGER型（整数）、REAL型（浮動小数点）、NUMERIC型（整数または浮動小数点）、NONE（変換しない）を指定することができます。



他にもプライマリキー制約やユニーク制約、NOT NULL制約等々といった、列に格納される値にさらに制限をかけることもここで行います。後程実際にその効果を確かめるコマンドを実行してみましょう。


SQLでは大文字小文字を区別しません。そのためどちらを使うのも自由ですが、基本的には視認性の観点からSQLコマンドは大文字、列名などは小文字にするのが一般的です。

```
CREATE TABLE test (a TEXT, b INTEGER, c TEXT)
```

列名には文字、数字、アンダースコアを用いることができますがハイフン '-' などを含めることはできません。その場合は列名をクォーテーションなどで括る必要があります。しかしクォーテーションの存在は面倒な事態を引き起こすことが多いため、省略するか別の文字で置換しておくことが無難でしょう。

今回は_に置き換えることにします。

では、列名を決めるためにcsvのヘッダ行を再確認しましょう。

In [ ]:
!head -n 1 eu.csv
# 最後の列のCOVID-19にハイフンが含まれているので次のセルではアンダースコアに置換

In [ ]:
%%sql
CREATE TABLE eu (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000);

In [ ]:
# 作成したテーブルが気に食わない、ミスしたなどで削除したい場合は
%sql DROP TABLE eu;

続いてcsvファイルの各行のデータから、テーブルへレコードをインサートするためのSQLクエリを生成します。レコードは1つずつインサートすることもできるのですが、まとめてインサートした方がトランザクションの関係上早く済みます。

#### レコードのインサート
テーブルにレコード（行）を記録するには、
```
INSERT INTO テーブル名 (列名1, 列名2, ...) VALUES (0, 1, 2, ...);
```
というコマンドを用います（テーブル名の後の列名は条件によっては省略できますがおすすめしません。テーブルの構造は後から変えることができるため、常にそのコマンドが意図した通りに実行されるとは限らないためです）。
```
INSERT INTO テーブル名 (列名1, 列名2, ...) VALUES (0, 1, 2, ...), (A, B, C, ...);
```
のように複数のレコードを指定することもできます。

colab上でレコードを作成しインサートするには、SQLクエリを直接文字列として作成したり、まずSQLクエリの枠だけ用意し、VALUES以降の値については？（プレースホルダ）としておき、後から実際に実行する時点で変数を渡す、などの方法があります。

早速試してみましょう。

In [ ]:
%%time
# pandasで作成する場合
eu = pd.read_csv('eu.csv', header=0)
tuples_pd = [tuple(row) for _i, row in eu.iterrows()]

display(len(tuples_pd))
display('-----')
display(tuples_pd[0])

In [ ]:
%%time
# pandasで作成する場合2
eu = pd.read_csv('eu.csv', header=0)
tuples_pd2 = eu.apply(tuple, axis=1)

display(len(tuples_pd2))
display(tuples_pd2[0])

In [ ]:
%%time
with open('eu.csv', 'r') as f:
  csv_reader = csv.reader(f)
  header = next(csv_reader)
  tuples_csv = [tuple(row) for row in csv_reader]

  display(len(tuples_csv))
  display(tuples_csv[0])

In [ ]:
# マジックコマンドで文字列のクエリを作りインサート
# 複数のレコードをインサートするように指示できますが、このデータを全て結合すると400万文字以上になるため、入力を受け付けてくれないでしょう。
# そこで効率は良くないですが１レコードずつインサートしてみます
# 問題点としては、文字列はクォーテーションで括らなければならず、もし元のデータにクォーテーションもしくはダブルクォーテーションが含まれていた場合は区別のために\を直前につける必要があることです（エスケープ）

In [16]:
num_of_dquot = !fgrep "\"" eu.csv

In [ ]:
len(num_of_dquot)
# 257件あるらしいことがわかる

In [ ]:
num_of_dquot # 内容のチェック　csvの区切り文字であるコンマを含む文字列があるための模様 この形式であれば上の操作をした時に除去されているため無視できる

In [ ]:
%%bash
fgrep "\'" eu.csv | head
# シングルクォートはないらしい

In [ ]:
# 文字列加工が望んだ結果になっているかをチェック
_t = tuples_csv[0]
quoted = ['\"' + x + '\"' for x in _t] # 各列の要素を"で囲む
_val = ','.join(quoted) # X, Y, Zのように繋げた文字列にする
display(_val)
cmd = 'INSERT INTO eu (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000) VALUES (' + _val + ')'
display(cmd)

In [ ]:
%%capture
for _t in tuples_csv[0:10000]: # 時間がかかるので10000件だけ 90秒ほど　出力が多いのでcaptureマジックコマンドを利用して抑制
  quoted = ['\"' + x + '\"' for x in _t] # 各列の要素を"で囲む
  _val = ','.join(quoted) # X, Y, Zのように繋げた文字列にする
  cmd = 'INSERT INTO eu (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000) VALUES (' + _val + ')'
  %sql $cmd


変数を利用した場合を試してみましょう。

In [ ]:
%%capture
%sql DELETE FROM eu;
for _t in tuples_csv[0:1000]: # 1000件だけ　12秒ほど
  dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative = _t
  %sql INSERT INTO eu (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000) VALUES \
  (:dateRep,:day,:month,:year,:cases,:deaths,:countriesAndTerritories,:geoId,:countryterritoryCode,:popData2019,:continentExp,:Cumulative)

変数を利用した方が楽に記述できそうですが、若干時間がかかるようです。

6万件のレコードをインサートするには12x60秒ほどかかると思われます。

#### バルクインサート
続いてバルクインサートを試してみましょう。バルクインサートも、先にSQL文を用意しておきますが、後から?に対応した値を格納したタプルをリストなどにしてまとめて渡します。
```
a = [0,1,2,3] # 配列
b = (0,1,2,3) # タプル　後から内容を変えることができない（イミュータブル）
```
イメージとしてはまさに先に挙げたインサートのようになります。
```
INSERT INTO テーブル名 (列名1, 列名2, ...) VALUES (0, 1, 2, ...);

INSERT INTO テーブル名 (列名1, 列名2, ...) VALUES (0, 1, 2, ...), (A, B, C, ...);
```
バルクインサートはマジックコマンド%sqlを用いた場合では挙動が不明なため、sqlite3ライブラリを用いて試します。これは1トランザクションでレコードのインサートを試みるため、先のループでインサートを繰り返すよりも高速化が見込めます。実行時間を比較するため、sqlite3ライブラリを用いて、1レコードずつforループでインサートするのも試してみましょう。

In [ ]:
# データベースを空に
%sql DELETE FROM eu

In [ ]:
%%time
# 1000レコードのみ試す
import sqlite3

con = sqlite3.connect('tmp.sqlite3')
cur = con.cursor()

cmd_create = 'CREATE TABLE IF NOT EXISTS eu (' \
  'dateRep TEXT,' \
  'day INTEGER,' \
  'month INTEGER,' \
  'year INTEGER,' \
  'cases INTEGER,' \
  'deaths INTEGER,' \
  'countriesAndTerritories TEXT,' \
  'geoId TEXT,' \
  'countryterritoryCode TEXT,'\
  'popData2019 TEXT,'\
  'continentExp TEXT,'\
  'Cumulative_number_for_14_days_of_COVID_19_cases_per_100000 REAL)'

cur.execute(cmd_create)
for _t in tuples_csv[0:1000]:
  cur.execute(
    'INSERT INTO eu (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000) '  \
    ' VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', _t
  )
  con.commit()
con.close()

7秒ほどかかりました。次にバルクインサートを試してみましょう。

forループで各タプルを渡す代わりに、タプルをリストでまとめているtuples_csvを直接渡しています（メソッドもexecuteからバルクインサート用のexecutemanyに変わっています）。

In [ ]:
! rm tmp.sqlite3

In [ ]:
%%time
import sqlite3
con = sqlite3.connect('tmp.sqlite3')
cur = con.cursor()

cmd_create = ('CREATE TABLE IF NOT EXISTS eu ('
  'dateRep TEXT,' 
  'day INTEGER,' 
  'month INTEGER,' 
  'year INTEGER,' 
  'cases INTEGER,' 
  'deaths INTEGER,' 
  'countriesAndTerritories TEXT,' 
  'geoId TEXT,' 
  'countryterritoryCode TEXT,'
  'popData2019 TEXT,'
  'continentExp TEXT,'
  'Cumulative_number_for_14_days_of_COVID_19_cases_per_100000 REAL)')
cur.execute(cmd_create)
cur.executemany(
    'INSERT INTO eu (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000) ' + \
    ' VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuples_csv
)

con.commit()
con.close()

In [ ]:
# どのようなテーブルが存在するか、どのようなコマンドで作られたかを確認する
%sql select * from sqlite_master;

### SQLによるデータ操作
それではテストデータが用意できたところでSQL文を用いてデータの操作をしてみましょう。

RDBの特徴として、行の順序や列の順序を重要視しないというものがあります。

そのため、X列Y行目という指定方法はありません。一応、テーブル定義の際の列の順序、インサートしたレコードの順序が、結果的に保たれている場合はありますが、それは保証されている動作ではないので、それを期待したコードは書かないのが無難です。

SQL言語の設計思想は、欲しいデータが持っている関係をプログラムに伝えれば、それを満たすようなデータが返ってくる（効率的に処理する役目は言語の方にある）、というものです。これによって、利用者側がデータ処理について考えることが減り、本来やりたいことであるデータ解析にリソースを割くことができます（というのが理想です、やはり実際にはある程度は内部動作を知っておく必要があります）。

pandasではqueryを用い、そこに条件式を書きました。SQLでは、

```
SELECT 列名1,... FROM テーブル名 [WHERE 条件式];
```
のようなSQL文で検索を行います。

試しに実行してみましょう。


In [ ]:
# cases列の指定
a = %sql SELECT cases FROM eu;

In [ ]:
# cases列の取得
%time res_cases = %sql SELECT cases from eu;
display(len(res_cases))

In [ ]:
# cases列の取得
%time %sql SELECT COUNT(cases) from eu;

In [ ]:
# cases < 60のレコードの取得 *は全ての列を意味
%time res_cases = %sql SELECT * from eu WHERE cases < 60;

In [ ]:
display(len(res_cases))

In [ ]:
# cases < 60のレコードの取得 *は全ての列を意味
%time res_cases = %sql SELECT * from eu WHERE cases == 58;

続けてもう少し大きいテーブルからの検索を実行してみます。

そのためにまずテーブルの用意を行います。先のバルクインサートで用いたtuples_csvを200回使って200倍のレコードをもつテーブルを作成してみます。もっと効率よくダミーデータを作成する手法もあったりするようなのですが、colab上のSQLiteで検証できていないのでこの単純な方法でデータを水増しします。

In [ ]:
%%time

con = sqlite3.connect('tmp.sqlite3')
cur = con.cursor()

cmd_create = ('CREATE TABLE IF NOT EXISTS eu2 ('
  'dateRep TEXT,' 
  'day INTEGER,' 
  'month INTEGER,' 
  'year INTEGER,' 
  'cases INTEGER,' 
  'deaths INTEGER,' 
  'countriesAndTerritories TEXT,' 
  'geoId TEXT,' 
  'countryterritoryCode TEXT,'
  'popData2019 TEXT,'
  'continentExp TEXT,'
  'Cumulative_number_for_14_days_of_COVID_19_cases_per_100000 REAL)')
cur.execute(cmd_create)
for i in range(0,200):
  cur.executemany(
      'INSERT INTO eu2 (dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID_19_cases_per_100000) ' + \
      ' VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuples_csv
  )

con.commit()
con.close()

In [ ]:
!ls -lha

In [ ]:
%sql SELECT COUNT(*) FROM eu2;

In [ ]:
%%time
# cases < 60のレコードの取得 *は全ての列を意味　文字列の場合はLIKE演算子など
res_cases = %sql SELECT cases from eu WHERE cases = 58;

In [ ]:
# cases列の取得
# RDBMSの方で処理が済んでも　pythonに送り込むときに大きく遅延することもある
#%time res_cases = %sql SELECT cases from eu;
#display(len(res_cases))

#### インデックスを作成し検索のスピードアップを期待する
pandasのインデックスは行のラベルという程度の意味でしたが、RDBのインデックスは文字通り索引のように検索を高速化させることに役立ちます（その仕組みについてはB-treeやT-treeについて調べてみてください）。

検索方法によっては高速化の恩恵にあずかれないこともありますが、データベースが大規模になる程効果を発揮します。

後からインデックスを追加することができますので、euテーブルのcases列にインデックスを追加してみます。

In [ ]:
%%sql
CREATE INDEX idx_cases ON eu (cases)

In [ ]:
%sql select * from sqlite_master;

In [ ]:
# インデックスを消したい場合
%sql DROP INDEX idx_cases

In [ ]:
%%time
# cases < 60のレコードの取得 *は全ての列を意味
res_cases = %sql SELECT cases from eu WHERE cases < 60;

In [ ]:
display(len(res_cases))

In [ ]:
# どのようにRDBMSがテーブルを検索しているかを知る
%sql EXPLAIN QUERY PLAN SELECT cases from eu WHERE cases < 60;

### pandas (python) と何が違う？
#### pandasでの処理が有利な点
操作するデータは全てメインメモリ上に存在する。
メインメモリへのアクセス速度はSSDやHDDへよりもはるかに高速である。
データの全ての要素に対してアクセスするような処理で、かつそのデータが全てメモリに載るサイズで、かつデータを操作するイベントが自分の管理下にある（自分が処理している間に他人や他のプログラムがデータにアクセスしない）という条件であれば、RDBよりもpython上で処理した方が良いケースが多いと思われる。

#### pandasでの処理が不利な点
* メモリに載らないサイズのデータはそもそも扱えないため、サイズダウンするか分割して処理する必要がある。
  * であればストレージの転送速度に拘束される。
* SSDやHDDに比べてメモリは容量あたりの価格が高いうえ、物理的な上限もありHPC向けのサーバーでも8TB程度までである。
* 現実的には数百GBを超えた時点でオンメモリ以外の方法を検討すべき（果たして本当に自分がやりたい処理はデータを全てメモリに乗せておく必要があるのか？）。
* 実質的にそのプログラムしかデータにアクセスすることができない
* データへのアクセスやデータ処理は全て自分でプログラミングする必要があり、操作効率は自身の実装に依存する

#### RDBが有利な点
* データ操作の信頼性を保つような性質 (ACID特性) を考慮したシステムが実装されており、データ操作の整合性を取るために自分が余計なことに気を使わなくて良い
 * A, Atomicity (不可分性):
 * C, Consistency (一貫性): 
 * I, Isolation (独立性): 
 * D, Durability (永続性): 
* データやデータ操作の具体的な内容に深く踏み込まず、抽象的なまま扱える
* どうやってデータを取得してくるのか、などデータ操作のためのプログラミングをしたり意識したりする必要がなく、どのようなデータが欲しいのかだけをSQL言語で記述すれば良い
  （とはいえ、パフォーマンスを出すためにはある程度内部構造を知っておく必要がある）
* 効率よくデータ操作できるように、RDBMSの方でオプティマイザがよしなに命令を解釈して実行してくれる
* データへのアクセス規約が整理されており、それに対応していればどのようなアプリケーションからでもデータにアクセスできる。

* メモリ上のデータは揮発するが、データを永続化し蓄積させることができる
* HDDのような遅い記憶装置に最適な操作が提供されているほか、効率的な検索のための機能がある（インデックスなど）
* そのためメモリに載せることができないようなサイズのデータ処理も可能


#### RDBが不利な点
* インデックスなどを活用しない場合、低速なストレージとデータをやり取りするため、高々数百MB, 数十GBのデータでは検索パフォーマンスはメモリ上で完結するpandasなどに比べて劣ると思われる
* インデックスを全く使わないのであればRDBMSを採用する意味があまりない
* 検索高速化のためにインデックスを作成すればするほど、データの更新や追記のコストが増大する（インデックスも更新する必要がある）
* ほとんどの場合、RDBMSはクライアント/サーバー構成を取るので、別マシンあるいは同一マシンの中でサーバーを起動しておく必要がある (SQLiteは単独で動作可能、dockerやsingularityで建てることができるので難しくはない)
* 基本操作だけならそれなりに平易とはいえ、SQL言語で記述する必要がある

## 具体的なデータを用いた練習

讃岐先生の授業で用いたデータのインポートなども試してみてください。

日本語を含むファイルは開くときに文字コードの問題が起きる可能性がありますが、Mac, linuxなら
```
iconv -f cp932 -t utf8 入力ファイル名 > 新規出力ファイル名
```
などとして文字コードを変換するか、
pd.read_csv(XXX, encoding='cp932')
などとして解決を試みてください。


インデックスを貼ってパフォーマンスを比較してみましょう。
基本的にインデックスは、その列のデータ行が多いほど、その列のデータの種類が多い（カーディナリティが高い）ほど有効になる。

例えば、入院か外来かの列にインデックスを張ってもあまり大した効果はないでしょう。

インデックスのデメリットとしては、データの追加・更新が遅くなることがあります。
それはインデックスの数が増えるほど顕著になりますが、それは更新のあった列などはインデックスを再計算することになるからです。
Microsoft SQL serverのエンタープライズ向けなど高額な商用ソフトであれば軽減するための機能があったりします。

In [ ]:
# いまどれほどメモリを消費しているか
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

## (参考: NoSQL) グラフデータベース

Reactomeなどのパスウェイデータベースが身近。SQLクエリとはまた別のCypher Query Language (CQL) を用いる。RDBよりも汎用性に欠けるが特定の目的にはRDBよりもパフォーマンスを発揮する。